# 「作法一」將爬下來的資料存進MongoDb

In [2]:
import requests
from datetime import datetime
import pandas as pd
from pymongo import MongoClient

In [6]:
#利用 Facebook Graph API Explorer 取得 token
#https://developers.facebook.com/tools/explorer
token = '<your_token>'

#將20個目標政治人物的粉專網址存在fanpage_urls
fanpage_urls = ['https://www.facebook.com/DoctorKoWJ/']

#取得粉絲專頁的id與名稱，並將其包成字典fanpage_dict
fanpage_dict = {}
for fanpage_url in fanpage_urls:
    resp = requests.get('https://graph.facebook.com/v2.12/{}?access_token={}&debug=all&format=json&method=get&pretty=0&suppress_http_code=1'
                       .format(fanpage_url,token))
    fanpage_json = resp.json()
    fanpage_dict[fanpage_json['id']] = fanpage_json['name']

#建立一個空的list          
posts = []

#使用for迴圈依序讀取粉絲頁的資訊，並使用.format()將fanpage_id與token傳入{}裡
#觀察發現Facebook Graph API回傳的data內最多包含100則貼文，故 limit = 100
for fanpage_id in fanpage_dict:
    fanpage_name = fanpage_dict[fanpage_id]
    resp = requests.get('https://graph.facebook.com/v2.9/{}/posts?limit=100&access_token={}'.format(fanpage_id, token))
    
    #使用while迴圈去取得所有的資料，並利用next翻頁
    while resp.json(): 
        for post_data in resp.json()['data']:
        # 利用ternary conditional operator判斷每個post是否含有'story'或'message'這些key，若無'story'則為純文字，無'message'則無文字
                story = post_data['story'] if 'story' in post_data else '純文字'
                message = post_data['message'] if 'message' in post_data else story
                created_time = datetime.strptime(post_data["created_time"].split('+')[0], "%Y-%m-%dT%H:%M:%S")
                post_id = post_data['id']
                posts.append({'fanpage_name':fanpage_name, 'message':message, 'created_time':created_time, 'story':story, 'id':post_id})
        if 'next' in resp.json()['paging']:
            resp = requests.get(resp.json()['paging']['next'])
        else:
            break

#最後將posts轉換成pd.DataFrame
posts_df = pd.DataFrame([post.values()for post in posts], columns=['粉絲專頁', '發文內容', '發文時間', '發文類型', 'post_id'])
posts_df

#將dataframe寫出為json，並存進MongoDb
client = MongoClient("mongodb://127.0.0.1:27017")
db = client['<db_name>']
collection = db["<collecion_name>"] 
collection.insert_many(posts)

In [7]:
posts_df

,粉絲專頁,發文內容,發文時間,發文類型,post_id
0,柯文哲,今天來開平餐飲學校學怎麼煎牛排，用英文學，希望最後有熟。\n\n#創新實驗教育\n臺北市教育...,2018-03-09 02:19:54,柯文哲 was live.,136845026417486_1263255723776405
1,柯文哲,【二備金不等於市長小金庫】\n\n目前的政府預算制度，大概在5、6月時就編列預算了，如果議會...,2018-03-08 09:00:00,純文字,136845026417486_1262597930508851
2,柯文哲,不同個性、不同能力的孩子，卻接受同樣一套教育，考同一張考卷，這並不合理。因此，「實驗教育」是...,2018-03-06 11:55:36,純文字,136845026417486_1260825347352776
3,柯文哲,今年的台北燈節落幕了，雖然資源有限，但市府同仁和策展單位都拼了，每一盞燈、每一座裝置藝術，我...,2018-03-05 11:00:00,純文字,136845026417486_1259930147442296
4,柯文哲,今晚，台北燈節就要告一段落，榮耀歸於所有辛苦的無名英雄們。\n\n很多公務員為了彩排昨晚的燈...,2018-03-04 11:58:40,柯文哲 added 4 new photos.,136845026417486_1259131887522122
5,柯文哲,過去公園裡的遊戲場，我們以為把跟罐頭一樣千篇一律的遊具擺上去就可以了，事實上玩這些遊具身體不...,2018-03-04 04:32:20,純文字,136845026417486_1258854180883226
6,柯文哲,今天晚上七點半，搭捷運來北門廣場，打開你的手機燈光，帶著你的小提燈，跟我一起在中華路點亮一隻...,2018-03-03 11:30:08,柯文哲 was live — at 北門廣場.,136845026417486_1258281140940530
7,柯文哲,【用燈節拉近世代距離】\n \n如果說世大運成功提升了台北人的光榮感，那麼在西區舉辦台北燈節...,2018-03-02 10:15:36,柯文哲 added 4 new photos.,136845026417486_1257206147714696
8,柯文哲,決定政策有三個要素：民意、專業、價值。進入網路時代之後，讓直接民主變成可能，iVoting就...,2018-03-01 09:43:11,純文字,136845026417486_1256313571137287
9,柯文哲,蔡總統、賴院長、二二八受難者與家屬、各位來賓、媒體朋友，大家好。\n\n1947年發生的二二...,2018-02-28 07:45:11,純文字,136845026417486_1255447907890520


# 「作法二」將爬下來的資料寫出為excel可以開啟的.csv

In [154]:
import requests
from datetime import datetime
import pandas as pd

In [9]:
#利用 Facebook Graph API Explorer 取得 token
token = 'your_token'

#將20個目標政治人物的粉專網址存在fanpage_urls
fanpage_urls = ['https://www.facebook.com/DoctorKoWJ/']

#取得粉絲專頁的id與名稱，並將其包成字典fanpage_dict
fanpage_dict = {}
for fanpage_url in fanpage_urls:
    resp = requests.get('https://graph.facebook.com/v2.12/{}?access_token={}&debug=all&format=json&method=get&pretty=0&suppress_http_code=1'
                       .format(fanpage_url,token))
    fanpage_json = resp.json()
    fanpage_dict[fanpage_json['id']] = fanpage_json['name']

#建立一個空的list          
posts = []

#使用for迴圈依序讀取粉絲頁的資訊，並使用.format()將fanpage_id與token傳入{}裡
#觀察發現Facebook Graph API回傳的data內最多包含100則貼文，故 limit = 100
for fanpage_id in fanpage_dict:
    fanpage_name = fanpage_dict[fanpage_id]
    resp = requests.get('https://graph.facebook.com/v2.9/{}/posts?limit=100&access_token={}'.format(fanpage_id, token))
    
    #使用while迴圈去取得所有的資料，並利用next翻頁
    while resp.json(): 
        for post_data in resp.json()['data']:
        # 利用ternary conditional operator判斷每個post是否含有'story'或'message'這些key，若無'story'則為純文字，無'message'則無文字
                story = post_data['story'] if 'story' in post_data else '純文字'
                message = post_data['message'] if 'message' in post_data else story
                created_time = datetime.strptime(post_data["created_time"].split('+')[0], "%Y-%m-%dT%H:%M:%S")
                posts.append([fanpage_name, message, created_time, story])

        if 'next' in resp.json()['paging']:
            resp = requests.get(resp.json()['paging']['next'])
        else:
            break


#最後將posts這個巢狀的list轉換成pd.DataFrame
posts_df = pd.DataFrame(posts, columns=['粉絲專頁', '發文內容', '發文時間', '發文類型']) 
posts_df

#將dataframe寫出為.csv
posts_df.to_csv('{}.csv'.format(posts[0][0]), index=False, encoding='utf-8') 

In [10]:
posts_df

,粉絲專頁,發文內容,發文時間,發文類型
0,柯文哲,今天來開平餐飲學校學怎麼煎牛排，用英文學，希望最後有熟。\n\n#創新實驗教育\n臺北市教育...,2018-03-09 02:19:54,柯文哲 was live.
1,柯文哲,【二備金不等於市長小金庫】\n\n目前的政府預算制度，大概在5、6月時就編列預算了，如果議會...,2018-03-08 09:00:00,純文字
2,柯文哲,不同個性、不同能力的孩子，卻接受同樣一套教育，考同一張考卷，這並不合理。因此，「實驗教育」是...,2018-03-06 11:55:36,純文字
3,柯文哲,今年的台北燈節落幕了，雖然資源有限，但市府同仁和策展單位都拼了，每一盞燈、每一座裝置藝術，我...,2018-03-05 11:00:00,純文字
4,柯文哲,今晚，台北燈節就要告一段落，榮耀歸於所有辛苦的無名英雄們。\n\n很多公務員為了彩排昨晚的燈...,2018-03-04 11:58:40,柯文哲 added 4 new photos.
5,柯文哲,過去公園裡的遊戲場，我們以為把跟罐頭一樣千篇一律的遊具擺上去就可以了，事實上玩這些遊具身體不...,2018-03-04 04:32:20,純文字
6,柯文哲,今天晚上七點半，搭捷運來北門廣場，打開你的手機燈光，帶著你的小提燈，跟我一起在中華路點亮一隻...,2018-03-03 11:30:08,柯文哲 was live — at 北門廣場.
7,柯文哲,【用燈節拉近世代距離】\n \n如果說世大運成功提升了台北人的光榮感，那麼在西區舉辦台北燈節...,2018-03-02 10:15:36,柯文哲 added 4 new photos.
8,柯文哲,決定政策有三個要素：民意、專業、價值。進入網路時代之後，讓直接民主變成可能，iVoting就...,2018-03-01 09:43:11,純文字
9,柯文哲,蔡總統、賴院長、二二八受難者與家屬、各位來賓、媒體朋友，大家好。\n\n1947年發生的二二...,2018-02-28 07:45:11,純文字


# References: